In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import pums

In [3]:
phx_pumas = ['0400112','0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400129']

In [4]:
# create a list of replicate weights
repwt = 'REPWT'
repwts = [repwt+str(i) for i in range(1, 81)]

In [5]:
df = pd.read_csv(f'../data/ipums_az_hhinc-size.csv.gz',compression='gzip')
df['GEO_ID'] = df.STATEFIP.astype(str).str.zfill(2) + df.PUMA.astype(str).str.zfill(5)

#phoenix pumas only and get rid of NA for HH  income
df = df[(df['GEO_ID'].isin(phx_pumas))&(df['HHINCOME']!=9999999)&(df['YEAR']!=2010)] #&df.HHINCOME!=9999999

#adjust for inflation (in 1999 dollars), weight income for total households
df['HHINCOME_99'] = df.HHINCOME * df.CPI99
df['INC_WT_99'] = df.HHWT * df.HHINCOME_99
df['INC_WT'] = df.HHWT * df.HHINCOME

#categorize by household size for AMI comparison
df['HHSize'] = df.NUMPREC.apply(lambda x: str(x) if x <= 8 else '9+' )

In [6]:
# new df reduced to necessary variables and group-by for HH size, PUMA, and year
p = df[['GEO_ID','HHWT','INC_WT','HHSize']+repwts].copy().groupby(['GEO_ID','HHSize']).sum().reset_index()

In [7]:
p['AVG_INC'] = p['INC_WT'] / p['HHWT']
p['hh_SE'] = p.apply(lambda x: (pums.get_se(x['HHWT'],x[repwts])),axis=1)
p['hh_MOE'] = p.apply(lambda x: (pums.get_moe(x['hh_SE'])),axis=1)
p['hh_CV'] = p.apply(lambda x: (pums.get_cv(x['HHWT'],x['hh_SE'])),axis=1)

In [8]:
p.head()

,GEO_ID,HHSize,HHWT,INC_WT,REPWT1,REPWT2,REPWT3,REPWT4,REPWT5,REPWT6,...,REPWT75,REPWT76,REPWT77,REPWT78,REPWT79,REPWT80,AVG_INC,hh_SE,hh_MOE,hh_CV
0,0400112,1,12269.0,8.193531e+08,12332,12914,13391,11923,12264,12647,...,11844,12444,12211,11191,11760,12023,66782.389763,779.228240,1281.830455,3.860909
1,0400112,2,51910.0,8.147022e+09,51294,52116,52094,52334,50808,50754,...,53148,53468,52068,50374,50662,53778,156945.138124,1951.618815,3210.412951,2.285483
2,0400112,3,30852.0,5.593913e+09,29544,27996,32094,29871,26565,29037,...,29319,29616,28851,28779,30867,30087,181314.446713,2646.625606,4353.699122,5.214868
3,0400112,4,26248.0,6.354897e+09,25956,26244,26180,26264,26204,25996,...,26148,25684,26036,27844,27984,25140,242109.775983,2091.447441,3440.431041,4.843785
4,0400112,5,8155.0,1.760520e+09,8780,7320,7545,9460,7460,7480,...,8660,6995,8075,7905,5965,8045,215882.335990,1598.991284,2630.340662,11.919450


In [9]:
p = p[['GEO_ID','HHSize','HHWT','AVG_INC','hh_MOE','hh_CV','INC_WT']].copy()

In [10]:
fin = pd.pivot_table(p,values=['AVG_INC','HHWT','hh_MOE','hh_CV','INC_WT'],\
                     index=['GEO_ID'],columns=['HHSize'],aggfunc=np.sum).reset_index()

In [11]:
fin

GEO_ID       AVG_INC                                               \
HHSize                      1              2              3              4   
0       0400112  66782.389763  156945.138124  181314.446713  242109.775983   
1       0400113  65803.281900  124776.364627  222802.549713  172451.807071   
2       0400114  49210.214207   95455.010256  105063.051080  109498.619883   
3       0400115  34374.038194   86768.313314   82234.027614  100432.395368   
4       0400116  55529.635160  100312.467423   87988.241835  106025.745426   
5       0400117  64736.375042  131248.386713  174604.034025  153824.117779   
6       0400118  40613.903187   88713.121954   99547.346397   56653.255419   
7       0400119  33967.531219   73395.101534   90784.292972   99434.057772   
8       0400120  74423.492123  133396.680928  131380.539185  143091.434857   
9       0400121  40733.898009   63535.587119   71604.629727   91035.907199   
10      0400122  30106.466451   52655.339015   77007.411624   65041.529968   
11      0400123  33741.619182   62622.303802   68502.847983   63579.207498   
12      0400125  39940.875685   76510.824181   98439.878244   81680.143549   
13      0400128  48170.131215  102929.769221  113697.143331  133126.178602   
14      0400129  72867.242198  121399.938182  172278.975871  190371.820910   

                                                                    \
HHSize              5              6              7              8   
0       215882.335990  213922.878229   86452.542373            NaN   
1       192191.951838  270945.948187            NaN            NaN   
2       105045.855821  146321.495327   84980.377358            NaN   
3       137077.288684  118484.128440   72706.953056   45000.000000   
4       145198.953537  195480.326797  134429.016189   56500.000000   
5       238184.904372   96532.945274   96410.655738            NaN   
6        99778.494624   87158.121331   73048.119048            NaN   
7        80686.602146   68424.065718   54179.185022  142062.549020   
8       179645.399147  154679.172714  103821.722846  149300.000000   
9        90049.847888   84021.194825  158225.551684   88361.403509   
10       58270.487239   92851.945461   76700.826613  117205.319149   
11       76871.266335   71959.732186   77591.068063   81500.000000   
12       76753.861205   45223.543994  175971.936460  137132.512156   
13      133758.639769   90262.597765   98410.625000            NaN   
14      178374.334491  141361.670395  293056.971678  258275.985130   

                      ...      hh_CV       hh_MOE                            \
HHSize            9+  ...         9+            1            2            3   
0                NaN  ...        NaN  1281.830455  3210.412951  4353.699122   
1       1.049500e+06  ...  43.198827  1528.509910  3142.168924  3005.422394   
2                NaN  ...        NaN  1589.735864  3533.755368  2445.453094   
3                NaN  ...        NaN  1479.524033  2744.859836  3569.824104   
4       3.942111e+05  ...  30.920397  1569.554813  2906.973236  2549.656300   
5       1.715504e+05  ...  33.473038  1584.514774  2785.284251  4009.366257   
6       1.006886e+05  ...  37.661115  1679.054933  3105.290207  3344.205854   
7       1.197862e+05  ...  32.810598  1045.553317  2312.914752  2808.030069   
8                NaN  ...        NaN  1327.466230  2939.579329  3050.637420   
9       2.121320e+05  ...  23.768656  1166.382810  2937.378440  4733.687525   
10      9.772410e+04  ...  25.230021   900.140859  1960.691848  3303.395057   
11      4.535113e+04  ...  17.492064   975.430732  2085.049408  3554.180011   
12      7.633826e+04  ...  30.518535  1590.289616  2900.166164  3600.102372   
13      2.070487e+05  ...  33.528399  1377.406888  3846.545256  3191.300753   
14      1.237664e+05  ...  33.049960  1275.199342  3313.693896  3754.704356   

                                                                         \
HHSize            4            5            6            7            8 

In [12]:
fin.to_excel('output/pums_inc_2021.xlsx')

In [25]:
test = df.copy().drop_duplicates()
test = test[['GEO_ID','HHWT','INC_WT']]
test = test.groupby(['GEO_ID']).sum().reset_index()

In [26]:
test

,GEO_ID,HHWT,INC_WT
0,0400112,58257.0,8.971028e+09
1,0400113,43132.0,5.779745e+09
2,0400114,47592.0,3.910546e+09
3,0400115,44062.0,3.293003e+09
4,0400116,50675.0,4.408771e+09
5,0400117,48019.0,5.656208e+09
6,0400118,49399.0,3.350234e+09
7,0400119,35433.0,2.626096e+09
8,0400120,42867.0,5.243230e+09
9,0400121,43773.0,3.381347e+09


In [28]:
test.to_excel('output/pums_test.xlsx')